In [1]:
import os
import logging
from logging.config import dictConfig

import requests
import pandas as pd
import pyarrow
from pyarrow import parquet


SEASONS = [
    "2014-15",
    "2015-16",
    "2016-17",
    "2017-18",
    "2018-19",
]
SEASON_TYPES = [
    "Regular+Season",
    "Playoffs"
]
LEAGUE_GAME_LOG_URL_TEMPLATE = "http://stats.nba.com/stats/leaguegamelog/?leagueId=00&season={}&seasonType={}&playerOrTeam=T&counter=0&sorter=DATE&direction=ASC&dateFrom=&dateTo="
PLAY_BY_PLAY_V2_URL_TEMPLATE = "http://stats.nba.com/stats/playbyplayv2/?gameId={}&startPeriod=0&endPeriod=14"
HEADERS = {
    'User-Agent': 'Some User Agent',
    'x-nba-stats-origin': 'stats',
    'Referer': 'https://stats.nba.com/',
}
DATA_PATH = "/home/matt/Desktop/nba_data"
LEAGUE_GAME_LOG_PATH = os.path.join(DATA_PATH, "raw", "leaguegamelog")
PLAY_BY_PLAY_V2_PATH = os.path.join(DATA_PATH, "raw", "playbyplayv2")


def configure_logger():
    logging_config = {'version': 1,
     'formatters': {'standard': {'format': '%(asctime)s %(module)s:%(lineno)d %(levelname)s %(message)s'}},
     'handlers': {'console': {'class': 'logging.StreamHandler',
       'level': 'DEBUG',
       'formatter': 'standard',
       'stream': 'ext://sys.stdout'}},
     'loggers': {'nba_stats_el': {'level': 'DEBUG', 'handlers': [], 'propagate': True}},
     'disable_existing_loggers': False,
     'root': {'level': 'WARNING', 'handlers': ['console']}}
    dictConfig(logging_config)
    return logging.getLogger('nba_stats_el')


logger = configure_logger()

In [2]:
def extract_load(seasons, season_types):
    for season in seasons:
        for season_type in season_types:
            leaguegamelog, playbyplayv2s = extract_season_season_type(season, season_type)
            logger.info(f"Loading leaguegamelog for {season} {season_type}")
            parquet.write_to_dataset(
                df_to_pq(leaguegamelog),
                LEAGUE_GAME_LOG_PATH
            )
            logger.info(f"Loading playbyplayv2 for {season} {season_type}")
            parquet.write_to_dataset(
                df_to_pq(playbyplayv2s),
                PLAY_BY_PLAY_V2_PATH,
                partition_cols=["SEASON", "SEASON_TYPE"]
            )


def df_to_pq(df):
    return pyarrow.Table.from_pandas(df, preserve_index=False)


def extract_season_season_type(season, season_type):
    logger.info(f"Extracting {season} {season_type}")
    leaguegamelog = extract_leaguegamelog(season, season_type)
    leaguegamelog['SEASON'] = season
    leaguegamelog['SEASON_TYPE'] = season_type
    game_ids = leaguegamelog['GAME_ID'].unique()
    playbyplayv2s = pd.concat([extract_playbyplayv2(game_id) for game_id in game_ids])
    playbyplayv2s['SEASON'] = season
    playbyplayv2s['SEASON_TYPE'] = season_type
    return leaguegamelog, playbyplayv2s


def extract_leaguegamelog(season, season_type):
    logger.info(f"Extracting leaguegamelog for {season} {season_type}")
    url = LEAGUE_GAME_LOG_URL_TEMPLATE.format(season, season_type)
    return nba_stats_request(url)


def extract_playbyplayv2(game_id):
    logger.info(f"Extracting playbyplayv2 for {game_id}")
    url = PLAY_BY_PLAY_V2_URL_TEMPLATE.format(game_id)
    return nba_stats_request(url)


def nba_stats_request(url):
    while True:
        try: 
            response = requests.get(url, headers=HEADERS, timeout=4)
        except requests.Timeout:
            logger.warning(f"Exceeded timeout, retrying {url}")
        else:
            break
    result = response.json()['resultSets'][0]
    columns = result['headers']
    data = result['rowSet']
    return pd.DataFrame(data, columns=columns)

In [3]:
extract_load(SEASONS, SEASON_TYPES)

2021-02-21 16:09:30,046 <ipython-input-2-3733157b9631>:23 INFO Extracting 2014-15 Regular+Season
2021-02-21 16:09:30,047 <ipython-input-2-3733157b9631>:35 INFO Extracting leaguegamelog for 2014-15 Regular+Season
2021-02-21 16:09:30,570 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400002
2021-02-21 16:09:31,009 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400003
2021-02-21 16:09:31,524 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400001
2021-02-21 16:09:32,158 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400010
2021-02-21 16:09:32,753 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400013
2021-02-21 16:09:33,263 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400006
2021-02-21 16:09:33,671 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400007
2021-02-21 16:09:34,076 <ipython-input-2-3733157b9631>:41 INFO Extracting 

2021-02-21 16:11:05,203 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0021400051&startPeriod=0&endPeriod=14
2021-02-21 16:11:09,307 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0021400051&startPeriod=0&endPeriod=14
2021-02-21 16:11:13,397 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0021400051&startPeriod=0&endPeriod=14
2021-02-21 16:11:17,516 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0021400051&startPeriod=0&endPeriod=14
2021-02-21 16:11:21,543 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0021400051&startPeriod=0&endPeriod=14
2021-02-21 16:11:25,691 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats

2021-02-21 16:13:07,082 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400083
2021-02-21 16:13:07,591 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400088
2021-02-21 16:13:07,993 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400084
2021-02-21 16:13:08,612 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400097
2021-02-21 16:13:12,636 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0021400097&startPeriod=0&endPeriod=14
2021-02-21 16:13:16,669 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0021400097&startPeriod=0&endPeriod=14
2021-02-21 16:13:20,694 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0021400097&startPeriod=0&endPeriod=14
2021-02-21 16:13:24,749 <ipython-input-2-3733157b9631>:

2021-02-21 16:15:39,550 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400104
2021-02-21 16:15:39,913 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400113
2021-02-21 16:15:40,371 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400116
2021-02-21 16:15:40,743 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400111
2021-02-21 16:15:44,981 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0021400111&startPeriod=0&endPeriod=14
2021-02-21 16:15:49,118 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0021400111&startPeriod=0&endPeriod=14
2021-02-21 16:15:53,256 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0021400111&startPeriod=0&endPeriod=14
2021-02-21 16:15:58,701 <ipython-input-2-3733157b9631>:

2021-02-21 16:18:08,336 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400135
2021-02-21 16:18:08,951 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400141
2021-02-21 16:18:09,365 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021400133
2021-02-21 16:18:13,392 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0021400133&startPeriod=0&endPeriod=14
2021-02-21 16:18:17,533 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0021400133&startPeriod=0&endPeriod=14
2021-02-21 16:18:21,560 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0021400133&startPeriod=0&endPeriod=14
2021-02-21 16:18:25,585 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0021400133&start

...

2021-02-21 20:06:34,072 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021801156
2021-02-21 20:06:34,598 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021801164
2021-02-21 20:06:36,119 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021801162
2021-02-21 20:06:36,649 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021801163
2021-02-21 20:06:37,975 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021801161
2021-02-21 20:06:38,492 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021801174
2021-02-21 20:06:39,200 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021801169
2021-02-21 20:06:39,722 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021801176
2021-02-21 20:06:40,167 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0021801166
2021-02-21 20:06:40,578 <ipython-input-2-3733157b9631>:41 INFO Extracting playbypl

2021-02-21 20:07:20,468 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800161
2021-02-21 20:07:21,002 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800101
2021-02-21 20:07:21,595 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800122
2021-02-21 20:07:22,099 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800142
2021-02-21 20:07:22,618 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800162
2021-02-21 20:07:23,238 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800152
2021-02-21 20:07:23,742 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800112
2021-02-21 20:07:24,252 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800132
2021-02-21 20:07:24,767 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800172
2021-02-21 20:07:28,916 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout

2021-02-21 20:09:26,718 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800211
2021-02-21 20:09:27,172 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800221
2021-02-21 20:09:27,741 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800201
2021-02-21 20:09:28,355 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800231
2021-02-21 20:09:28,865 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800212
2021-02-21 20:09:29,386 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800222
2021-02-21 20:09:29,794 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800202
2021-02-21 20:09:30,306 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800232
2021-02-21 20:09:30,820 <ipython-input-2-3733157b9631>:41 INFO Extracting playbyplayv2 for 0041800213
2021-02-21 20:09:31,332 <ipython-input-2-3733157b9631>:41 INFO Extracting playbypl

2021-02-21 20:12:17,470 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0041800214&startPeriod=0&endPeriod=14
2021-02-21 20:12:21,640 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0041800214&startPeriod=0&endPeriod=14
2021-02-21 20:12:25,997 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0041800214&startPeriod=0&endPeriod=14
2021-02-21 20:12:30,137 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0041800214&startPeriod=0&endPeriod=14
2021-02-21 20:12:34,305 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0041800214&startPeriod=0&endPeriod=14
2021-02-21 20:12:38,337 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats

2021-02-21 20:15:14,493 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0041800216&startPeriod=0&endPeriod=14
2021-02-21 20:15:18,629 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0041800216&startPeriod=0&endPeriod=14
2021-02-21 20:15:22,780 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0041800216&startPeriod=0&endPeriod=14
2021-02-21 20:15:26,905 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0041800216&startPeriod=0&endPeriod=14
2021-02-21 20:15:31,070 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats.nba.com/stats/playbyplayv2/?gameId=0041800216&startPeriod=0&endPeriod=14
2021-02-21 20:15:35,099 <ipython-input-2-3733157b9631>:51 WARNING Exceeded timeout, retrying http://stats